# Assignment

## Importing Libraries

In [2]:
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import seaborn as sns

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Currently using {device}')

Currently using cpu


## Loading the Datasets

In [3]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

cifar_dataset_train = datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)

cifar_dataset_test = datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
classes = ()

## Creating the training dataset

In [5]:
from torch.utils.data import DataLoader
batch_size = 100

train_iter = DataLoader(cifar_dataset_train,batch_size,shuffle=True,num_workers=2)

In [6]:
X,y = next(iter(train_iter))

## Creating the test dataset

In [7]:
test_iter = DataLoader(cifar_dataset_test,batch_size,shuffle=False,num_workers=2)

In [8]:
X_test,y_test = next(iter(test_iter))

## Analysing the CIFAR-10 data

In [9]:
print(f'The dimensions of each input image is {X.size()} The number of input channels is {X.size()[1]}')
print(f'The output labels are as follows {y}')

The dimensions of each input image is torch.Size([100, 3, 32, 32]) The number of input channels is 3
The output labels are as follows tensor([8, 9, 1, 6, 3, 8, 0, 7, 7, 7, 1, 7, 7, 8, 1, 0, 2, 3, 2, 4, 9, 2, 6, 2,
        3, 0, 7, 5, 7, 6, 1, 4, 1, 3, 8, 7, 3, 5, 7, 3, 8, 0, 0, 7, 5, 0, 3, 4,
        0, 8, 3, 1, 7, 4, 1, 2, 2, 6, 9, 2, 9, 1, 3, 5, 3, 2, 6, 6, 9, 5, 0, 6,
        0, 9, 3, 7, 8, 1, 7, 2, 2, 5, 3, 8, 0, 5, 3, 1, 4, 9, 6, 2, 9, 7, 2, 1,
        5, 4, 9, 9])


## Example image

## CNN Architecture

In [187]:
class Block(torch.nn.Module):
    def __init__(self,input_channels,output_channels,strides,features,out_features,use_1x1conv=False):
        super(Block,self).__init__()
        self.output_channels = output_channels

        self.b1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=input_channels,out_channels=output_channels,kernel_size=3,stride=strides,padding=1),
            torch.nn.BatchNorm2d(output_channels),
            torch.nn.ReLU(),
        )

        self.b2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=input_channels,out_channels=output_channels,kernel_size=5,stride=strides,padding=2),
            torch.nn.BatchNorm2d(output_channels),
            torch.nn.ReLU(),
        )

        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=features,out_features=out_features),
            torch.nn.ReLU(),
            torch.nn.Dropout(0)
        )

        if use_1x1conv:
            self.conv = torch.nn.Conv2d(input_channels,output_channels,kernel_size=1,stride=strides)

        else:
            self.conv = None

    def forward(self,x):

        b1 = self.b1(x)
        b2 = self.b2(x)

        b = torch.concat((b1.view(100,-1),b2.view(100,-1)),dim=1)

        linear = self.linear(torch.flatten(x))

        out = torch.matmul(linear,b.view(2,-1))

        if self.conv:
            x = self.conv(x)

        out = torch.nn.functional.relu(x + out.view(100,self.output_channels,x.size(2),-1))

        return out

In [188]:
class CNN(torch.nn.Module):
    def __init__(self,num_classes):
        super(CNN,self).__init__()

        self.stem = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d((24,24))
        )

        self.block1 = Block(input_channels=3,output_channels=64,features=172800,out_features=2,strides=1,use_1x1conv=True)
        self.block2 = Block(input_channels=64,output_channels=128,features=3686400,out_features=2,strides=1,use_1x1conv=True)

        self.last = torch.nn.Sequential(torch.nn.AdaptiveAvgPool1d(100),
                                        torch.nn.Flatten(),
                                        torch.nn.Linear(in_features=12800,out_features=100),
                                        torch.nn.ReLU(),
                                        torch.nn.Dropout(0)
                                        )

        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=100,out_features=num_classes),
            torch.nn.ReLU(),
            torch.nn.Dropout(0),
            torch.nn.Softmax(dim=1)
        )

    def forward(self,x):

        output = self.stem(x)

        output = self.block1(output)
        output = self.block2(output)

        output = self.last(output.view(100,128,-1))

        output = self.linear(output)

        return output

In [189]:
model = CNN(num_classes=10).to(device)
print(model)

CNN(
  (stem): Sequential(
    (0): AdaptiveAvgPool2d(output_size=(24, 24))
  )
  (block1): Block(
    (b1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (b2): Sequential(
      (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (linear): Sequential(
      (0): Linear(in_features=172800, out_features=2, bias=True)
      (1): ReLU()
      (2): Dropout(p=0, inplace=False)
    )
    (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (block2): Block(
    (b1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (b2): Sequential(

In [190]:
model(X).size()

torch.Size([100, 10])

## Create Loss and Optimization

In [191]:
loss = torch.nn.CrossEntropyLoss()

# Create the optimizer term
optimizer = torch.optim.SGD(model.parameters(), lr=0.005,weight_decay=0.005,momentum=0.9)

## Calculate Loss, and Validation Accuracy

In [192]:
def train_test_model(model,train_iter,test_iter,epochs,optimizer,loss):
    for epoch in range(epochs): # Initialize the epochs

        l_score = []

        for i,(X,y) in enumerate(train_iter):

            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()

            #model.train()
            y_hat = model(X)
            l = loss(y_hat,y)
            l.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {l.item():.4f}')

        with torch.no_grad():
            correct = 0
            total = 0
            for data in test_iter:
                X_test,y_test = data
                X_test = X_test.to(device)
                y_test = y_test.to(device)

                y_hat = model(X_test)

                _, predicted = torch.max(y_hat.data,1)
                total += y_test.size(0)
                correct += (predicted == y_test).sum().item()

            print(f'Accuracy of the network on the test images is : {100 * correct // total} percent')

    return

In [193]:
epochs = 10
train_test_model(model,train_iter,test_iter,epochs,optimizer,loss)

KeyboardInterrupt: 